In [1]:
from gwpy.timeseries import TimeSeries
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from gwpy.time import tconvert
from scipy.signal import find_peaks
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
import pandas as pd

In [ ]:
#define parameters
start_time = 1262217618  #Jan 5, 2020 - Jan 12, 2020
duration = 24*3600*8
channels = ['ETMX_Z','ITMY_Z','ETMY_Z','ETMX_X','ITMY_Y','ETMY_Y']

peakwidth = 5
peakprom = 100
peakdist = 60

In [ ]:
%%time
#get time series in microseismic band ground motion
channelRMS = []
for chn in channels:
    chname = 'L1:ISI-GND_STS_' + chn + '_BLRMS_100M_300M.max'
    data = TimeSeries.find(chname, start_time, start_time + duration, frametype ='L1_M', verbose=True)
    channelRMS.append(data.value)

In [ ]:
#store data
eqdata100m300m = pd.DataFrame(np.transpose(channelRMS), columns = [channels])
eqdata100m300m

In [ ]:
#save to csv
eqdata100m300m.to_csv("eqdata_jan5-12_100M_300M.csv") 

In [ ]:
eqdatadf = pd.read_csv("eqdata_jan5-12_100M_300M.csv")

In [ ]:
%%time
#find time series for lockbit data
Lockbit = TimeSeries.find('L1:GRD-ISC_LOCK_STATE_N.min',start_time, start_time + duration, frametype ='L1_M', verbose=True)

In [ ]:
np.unique(Lockbit.value)

In [ ]:
#find peaks just using a threshold
peaks,_ = find_peaks(channelRMS[0],height = 100)
peaks

In [ ]:
#plot peaks in ETMX_Z
plt.figure(figsize=(12,8))
plt.plot(channelRMS[0])
plt.plot(peaks, channelRMS[0][peaks],'*')
plt.yscale('log')
plt.show()

In [ ]:
#find peaks using parameters
peaks,_ = find_peaks(channelRMS[0], width = peakwidth, distance = peakdist, prominence = peakprom)
peaks

In [ ]:
#plot peaks in ETMX_Z with parameters
plt.figure(figsize=(12,8))
plt.plot(channelRMS[0])
plt.plot(peaks, channelRMS[0][peaks],'*')
plt.yscale('log')
plt.show()

In [ ]:
#plot peaks in ETMXZ, ITMY_Z,AND ETMY_Z with parameters
plt.figure(figsize=(12,8))
peaksRMSix = [] #location of local peaks in each Z- channel

for jj in range(3):
    peaksix,_ = find_peaks(channelRMS[jj], width = peakwidth, distance = peakdist, prominence = peakprom)
    peaksRMSix.append(peaksix)
    plt.plot(channelRMS[jj])
    plt.plot(peaksix, channelRMS[jj][peaksix],'*')

plt.yscale('log')
plt.show()

In [ ]:
#find common peaks in at least two of the three Z-channels, define those as EQs
EQix = []

EQix.append(np.intersect1d(peaksRMSix[0],peaksRMSix[1]))
EQix.append(np.intersect1d(peaksRMSix[1],peaksRMSix[2]))
EQix.append(np.intersect1d(peaksRMSix[2],peaksRMSix[0]))


EQix = np.unique(np.concatenate(EQix))
EQix

In [ ]:
#find if detector survived EQ or lost lock status
survived = []
lostlock = []
delta = 10  #minutes - interval before and after EQ to define survival

for jj in EQix:
    if np.min(Lockbit.value[jj-delta:jj+delta])>=2000:
        survived.append(jj)
    else:
        lostlock.append(jj)